In [7]:
import pandas as pd
import re
from rapidfuzz import fuzz, process

In [ ]:
# Módulo TextCleanerAndMatchFinder: 
"""
Descripción del módulo:
Este módulo realiza la limpieza y normalización de texto en las columnas de un DataFrame, específicamente en las descripciones de productos y proveedores. 
Luego, utiliza la librería RapidFuzz para encontrar coincidencias entre las descripciones de productos y proveedores basadas en su similitud.
"""

# Función de limpieza de texto para una columna específica del DataFrame
def clear_column(column):
    """
    Limpia y normaliza el texto en una columna para eliminar términos irrelevantes y uniformar el formato.
    
    Elimina:
    - Números, unidades y abreviaturas como 'mg', 'ml', 'g', etc.
    - Caracteres especiales como puntos, comas, guiones, etc.
    - Espacios extra y convierte todo a minúsculas.
    - Palabras comunes como 'de', 'la', 'el', 'a', etc.
    - Palabras repetidas y términos relacionados con la variabilidad en dosis/presentación.
    
    Parámetros:
        column (str): Texto a limpiar.

    Retorna:
        str: Texto limpio y normalizado.
    """
    columna_limpia = re.sub(
        r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
        r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', 
        '', 
        column, 
        flags=re.IGNORECASE
    )
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()
    columna_limpia = columna_limpia.lower()
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|por|medicamento|tratamiento|ac)\b', '', columna_limpia).strip()
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)
    return columna_limpia

# Función para cargar los datos desde un archivo CSV
def load_data(df_archivo):
    """
    Carga un archivo CSV en un DataFrame de pandas.
    
    Parámetros:
        df_archivo (str): Ruta o nombre del archivo CSV.
    
    Retorna:
        DataFrame: DataFrame con los datos cargados, o None si hubo un error.
    """
    try:
        df = pd.read_csv(df_archivo)
        return df
    except Exception as e:
        print(f"Error al cargar el archivo {df_archivo}: {e}")
        return None

# Cargar los datos desde archivos CSV
name_df_provedor = "prueba.csv"  
name_df_productos = "L_STMaestroPrd.csv"  

# Cargar los DataFrames
df_proveedor = load_data(name_df_provedor)
df_productos = load_data(name_df_productos)

# Copias de los DataFrames
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()  # Eliminar columnas vacías
copy_dfProveedores = df_proveedor.copy()

# Limpiar las descripciones en ambas tablas
copy_dfProductos['descrip1_limpia'] = copy_dfProductos['descrip1'].fillna('').apply(clear_column)
copy_dfProveedores['descripcion_limpia'] = copy_dfProveedores['DESCRIPCIÓN'].fillna('').apply(clear_column)

# Crear índice para búsqueda rápida
proveedor_index = copy_dfProveedores['descripcion_limpia'].tolist()




# Función para encontrar la mejor coincidencia entre descripciones de productos y proveedores
def find_best_match(product_desc, product_row, proveedor_index, proveedor_df, threshold=80):
    """
    Encuentra la mejor coincidencia de una descripción de producto entre las descripciones de proveedores
    utilizando la librería RapidFuzz.
    
    Parámetros:
        product_desc (str): Descripción del producto.
        product_row (DataFrame row): Fila del producto con todas sus columnas.
        proveedor_index (list): Lista de descripciones limpias de proveedores.
        proveedor_df (DataFrame): DataFrame con las descripciones de proveedores.
        threshold (int): Umbral de similitud mínima para considerar una coincidencia.
    
    Retorna:
        tuple: Un diccionario con las columnas del producto, proveedor y la similitud, o None si no hay coincidencia.
    """
    best_match = process.extractOne(product_desc, proveedor_index, scorer=fuzz.ratio)
    if best_match and best_match[1] >= threshold:
        # Encontrar la fila del proveedor con la mejor coincidencia
        matched_row = proveedor_df[proveedor_df['descripcion_limpia'] == best_match[0]].iloc[0]
        
        # Combinar la fila del producto con la fila del proveedor y agregar la similitud
        match_info = product_row.to_dict()  # Convertir la fila del producto en un diccionario
        match_info.update(matched_row.to_dict())  # Actualizar con la fila del proveedor
        match_info['similaridad'] = best_match[1]  # Agregar la similitud
        
        return match_info
    return None

# Comparar las descripciones de los productos con los proveedores
matches = []
no_matches = []

# Iterar por cada producto y encontrar su mejor coincidencia
for _, prod_row in copy_dfProductos.iterrows():
    prod_desc = prod_row['descrip1_limpia']
    match_info = find_best_match(prod_desc, prod_row, proveedor_index, copy_dfProveedores)
    if match_info:
        matches.append(match_info)
    else:
        no_matches.append(prod_desc)

# Convertir los resultados en DataFrames
df_matches = pd.DataFrame(matches)
df_no_matches = pd.DataFrame(no_matches, columns=['producto_sin_match'])

# Reordenar columnas para que la columna 'similaridad' esté al principio
cols = ['similaridad'] + [col for col in df_matches.columns if col != 'similaridad']
df_matches = df_matches[cols]

# Mostrar los resultados
print(f"Coincidencias encontradas: {len(df_matches)}")
print(df_matches)

print(f"\nProductos sin coincidencias: {len(df_no_matches)}")
print(df_no_matches)


C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_11524\601458004.py:53: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_archivo)


Coincidencias encontradas: 105
     similaridad    niprod cod_prod cod_abrev  \
0     100.000000   98611.0    31128     31128   
1     100.000000   14745.0    11631     11631   
2     100.000000   14746.0    11630     11630   
3      84.210526  113090.0     1177      1177   
4      84.210526  113091.0    51966     51966   
..           ...       ...      ...       ...   
100    91.228070   91647.0     6839      6839   
101    91.228070   96762.0    10069     10069   
102    82.352941  107037.0    27370     27370   
103    82.352941  107038.0    30569     30569   
104    82.352941  105725.0    45577     45577   

                                           descrip1  \
0                    ASPIRINA 4 A 500 mg comp.x 100   
1                      ASPIRINA G 500 mg comp.x 100   
2                       ASPIRINA G 500 mg comp.x 20   
3                            ASPIRINETAS comp.x 100   
4                             ASPIRINETAS comp.x 28   
..                                              ..

In [15]:
df_matches

,similaridad,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,...,atrib1,atrib2,descrip1_limpia,EAN,DESCRIPCIÓN,PVP,DROGUERIA,DTO.,CANT.X BULTO,descripcion_limpia
0,100.000000,98611.0,31128,31128,ASPIRINA 4 A 500 mg comp.x 100,NaN,ASPIRINA 4 A 500 mg,PT,NaN,NaN,...,NaN,NaN,aspirina,7791939000089,ASPIRINA 100\r\nEstuche: 10 blisters x 10 comp...,"$4.904,38","$2.305,06",53%,50,aspirina
1,100.000000,14745.0,11631,11631,ASPIRINA G 500 mg comp.x 100,NaN,ASPIRINA G 500 mg co,PT,7795360000571,NaN,...,NaN,NaN,aspirina,7791939000089,ASPIRINA 100\r\nEstuche: 10 blisters x 10 comp...,"$4.904,38","$2.305,06",53%,50,aspirina
2,100.000000,14746.0,11630,11630,ASPIRINA G 500 mg comp.x 20,NaN,ASPIRINA G 500 mg co,PT,7795360000564,NaN,...,NaN,NaN,aspirina,7791939000089,ASPIRINA 100\r\nEstuche: 10 blisters x 10 comp...,"$4.904,38","$2.305,06",53%,50,aspirina
3,84.210526,113090.0,1177,1177,ASPIRINETAS comp.x 100,NaN,ASPIRINETAS comp.x 1,PT,7793640210191,NaN,...,NaN,NaN,aspirinetas,7791939000089,ASPIRINA 100\r\nEstuche: 10 blisters x 10 comp...,"$4.904,38","$2.305,06",53%,50,aspirina
4,84.210526,113091.0,51966,51966,ASPIRINETAS comp.x 28,NaN,ASPIRINETAS comp.x 2,PT,7793640000839,NaN,...,NaN,NaN,aspirinetas,7791939000089,ASPIRINA 100\r\nEstuche: 10 blisters x 10 comp...,"$4.904,38","$2.305,06",53%,50,aspirina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,91.228070,91647.0,6839,6839,SALES PARA REHIDRATACION ORAL sob.x 3 x 27.5 g,NaN,SALES PARA REHIDRATA,PT,7793213503460.0,NaN,...,NaN,NaN,sales rehidratacion oral sob,7791939000171,SALES REHIDRATACION ORAL Estu...,"$9.318,90","$4.379,88",53%,60,sales rehidratacion oral grs
101,91.228070,96762.0,10069,10069,SALES PARA REHIDRATACION ORAL sob.x 3 x 27.9 g,CAJA X 3 SOBRES. UM: SOBRE,SALES PARA REHIDRATA,PT,7791939000171.0,NaN,...,NaN,NaN,sales rehidratacion oral sob,7791939000171,SALES REHIDRATACION ORAL Estu...,"$9.318,90","$4.379,88",53%,60,sales rehidratacion oral grs
102,82.352941,107037.0,27370,27370,SPIRULINA A.M. 300 mg comp.x 100,NaN,SPIRULINA A.M. 300 m,PT,NaN,NaN,...,NaN,NaN,spirulina,7791939000089,ASPIRINA 100\r\nEstuche: 10 blisters x 10 comp...,"$4.904,38","$2.305,06",53%,50,aspirina
103,82.352941,107038.0,30569,30569,SPIRULINA A.M. 500 mg comp.x 60,NaN,SPIRULINA A.M. 500 m,PTO,NaN,NaN,...,NaN,NaN,spirulina,7791939000089,ASPIRINA 100\r\nEstuche: 10 blisters x 10 comp...,"$4.904,38","$2.305,06",53%,50,aspirina


In [11]:
df_no_matches

,producto_sin_match
0,
1,
2,repetido supportan
3,bolsas papel grado medico 175x65x420mm kims
4,bolsas papel grado medico 260x60x520mm kims
...,...
56664,zyvalix
56665,zyvalix plus
56666,zyvox iv bolsas 10x300ml
56667,zyvox oral tab
